In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import seaborn as sb
import calendar
import datetime as dt

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go

from pylab import rcParams
rcParams['figure.figsize'] = 35, 40

import fnmatch
import os
from tqdm import tqdm

## Читаем данные

In [73]:
data=pd.read_pickle('MG_Sales.pickle',compression='gzip')

In [74]:
#получаем колонки категориальных характеристик
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']

In [75]:
#заполняем пустые значения в категориальных данных
data[categorical_columns]=data[categorical_columns].applymap(lambda cell: cell if bool(cell.strip()) else '<Неопределено>')

## Визуализируем данные

In [134]:
#data[data['Год']==2016].pivot_table(index=['Месяц','День'], columns='ТоварЦеноваяГруппа', values='Количество', aggfunc=sum,fill_value=0,dropna=False).plot( legend=True,stacked=True)
data_table=data.pivot_table(index=['Год','Месяц','День'], columns='ТоварТип', values='Количество', aggfunc=sum,fill_value=0,dropna=False)

def convertData(ymd):
    y=ymd.name[0]
    m=ymd.name[1]
    d=ymd.name[2]
    try: 
        if (m==2)&(d==29): 
            return dt.datetime(1980,1,1)
        else:
            return dt.datetime(y,m,d) 
    except ValueError: 
        return dt.datetime(1980,1,1)

#преобразуем дату
data_table['Дата']=data_table.apply(convertData, axis=1)

#добавляем колонки с датами
jobDay=data[data['РабочиеВыходные']>1].groupby(['Год','Месяц','День'])['РабочиеВыходные'].max()*100
celebrate=data[data['Праздник']==1].groupby(['Год','Месяц','День'])['Праздник'].max()*300

#соединяем все
data_table=pd.concat([data_table, jobDay,celebrate], axis=1).reset_index()#, join='inner'

#удаляем лишнее
data_table.drop(['Год','Месяц','День'], axis=1, inplace=True)
data_table.drop(data_table[data_table['Дата']==dt.datetime(1980,1,1)].index, inplace=True)
data_table.drop(data_table[data_table['Дата']>=dt.datetime.now()].index, inplace=True)

In [135]:
init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        if column in ['РабочиеВыходные','Праздник']:
            trace = go.Bar(
                x = df.index,
                y = df[column],                
                name = column,
                opacity=0.6,
                marker=dict(
                    color='rgba(96, 96, 96, 0.6)',
                    line=dict(
                        color='rgba(96, 96, 96, 0.6)',
                        width=1,
                    )
                )   
            )
        else:
            trace = go.Scatter(
                x = df.index,
                y = df[column],
                mode = 'lines',            
                name = column,
                line=dict(
                    shape='spline'
                )
            )
                        
        data.append(trace)
    
    layout = dict(title = title
    )
    
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)
    

plotly_df(data_table.set_index('Дата'))

In [133]:
data.columns

Index(['АртикулБезКачества', 'Вес', 'ВидДизайна', 'ВидИзделия', 'Год',
       'ГруппаТовара', 'День', 'ДеньНедели', 'Количество', 'Коллекция',
       'Контрагент', 'МаркетинговаяЛинейка', 'Месяц',
       'НеЗаказыватьУПоставщиков', 'Неделя', 'ОсновнойКамень', 'ПодвидДизайна',
       'ПодвидИзделия', 'Праздник', 'РабочиеВыходные', 'Размер',
       'СтильДизайна', 'Товар', 'ТоварСреднийВес', 'ТоварТип',
       'ТоварЦеноваяГруппа', 'ЦветМеталла', 'ЦветПокрытия'],
      dtype='object')

In [60]:
data_table=data.pivot_table(index=['Год','Месяц','День'], columns=['ЦветМеталла'], values='Количество', aggfunc=sum,fill_value=0,dropna=False)

In [68]:
data_table=data.pivot_table(index=['Год','Месяц','День'], columns=['ЦветМеталла'], values='Количество', aggfunc=sum,fill_value=0,dropna=False)
weekDay=data.groupby(['Год','Месяц','День'])['ДеньНедели'].max()
jobDay=data.groupby(['Год','Месяц','День'])['РабочиеВыходные'].max()
celebrate=data.groupby(['Год','Месяц','День'])['Праздник'].max()

data_table=pd.concat([data_table, weekDay,jobDay,celebrate], axis=1, join='inner').reset_index()

In [69]:
data_table

,Год,Месяц,День,<>,<Неопределено>,Белое золото,Жёлтое золото,Жёлтое/белое золото,Золотое покрытие,Красное золото,...,Красное/белое/жёлтое зол.,Платина,Прочее,Серебро,Серебро с позолотой,Серебро с черным родием,Ювелирный сплав,ДеньНедели,РабочиеВыходные,Праздник
0,2014,1,1,0,0,10,2,0,0,75,...,0,0,6,27,0,0,2,2,1.0,1.0
1,2014,1,2,0,0,89,4,1,0,461,...,0,0,9,72,0,0,33,3,1.0,1.0
2,2014,1,3,0,1,77,7,0,0,485,...,0,0,12,147,0,0,40,4,2.0,1.0
3,2014,1,4,0,0,57,7,0,0,437,...,0,0,6,170,0,0,29,5,3.0,1.0
4,2014,1,5,0,0,75,11,0,0,464,...,0,0,4,117,0,0,54,6,3.0,1.0
5,2014,1,6,0,0,97,2,0,0,487,...,0,0,8,155,0,0,43,0,1.0,1.0
6,2014,1,7,0,0,78,2,0,0,388,...,0,0,13,58,0,0,19,1,1.0,1.0
7,2014,1,8,0,0,52,1,0,0,339,...,0,0,2,80,0,0,19,2,1.0,1.0
8,2014,1,9,0,0,59,4,0,0,397,...,0,0,6,104,0,0,18,3,1.0,-1.0
9,2014,1,10,0,0,66,5,0,0,447,...,0,0,12,136,0,0,28,4,2.0,-1.0
